In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# COVID-19, School Closures and Learning Situation

# Introduction
We are facing a COVID-19 education crisis. schools for more than 168 million children globally have been closed for almost one and half year. With every day that goes by, these children will fall further behind and the most vulnerable will pay the heaviest price. Governments must reopen schools better than before and ensure every childcan return to school.

There are many variables that are considered as factors and indicators in the evolution of school closures, like countries incomes class, vaccinations rate, population, new deaths and new cases COVID-19 rate, the data that I found here, says otherwise.

What makes countries open or close their schools?

When this online learning happens, which students need help the most?


This analysis divided to two sections:
 
1. Analysis Overview
2. Data Overview
3. Pre-Processing Data
4. Statistical Analysis adn Data Visualization
5. Conclusions

In [ ]:
!pip install xlrd
!pip install openpyxl
!pip install pycountry_convert


In [ ]:
import matplotlib 
import seaborn as sns
import statsmodels as sm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopy as gpy
import geopandas as gpd
from warnings import filterwarnings
import pycountry_convert
import os
import pycountry_convert as pc



# ANALYSIS OVERVIEW

**This analysis has two chapters**
1. Covid-19 and School Closures
2. Covid-19 and its Impact on Student

**School closures and COVID-19:**
1. Where we stand today?
2. The evolution of school closures
3. The relationships betweens school closures, vaccinations, and countries economies class in current situations
4. Government policy is still the biggest factor of school closures evolutions, followed with Economy, Vaccinations and Population

**Dependent Variables:**
1. School Closures

**Independent Variable:**
1. Countries
2. Vaccinations
3. Income Classification
4. Population
5. Population Density

**Learning Situation and Behavior in New Delhi, Covid-19 Period:**
1. Current learning situations
2. Students sleep behavior
3. Students prefer to self learning than distraction

**Independent Variable:**
1. Level of School

**Dependent Variables:**
1. Time spent on Online Class
2. Time spent on Self Study
3. Time spent on sleep
4. time spent on social media
5. Rating on Online Class
6. Is student has enough time of study?
7. Is student has normal time of sleep?
8. Is student spent more time for social media?

# Data Overview

1. Global school closures COVID-19 by UNESCO
(https://data.humdata.org/dataset/global-school-closures-covid19)

2. Covid vaccinations
(https://ourworldindata.org/covid-vaccinations)

3. New World Bank country classifications by income level: 2021-2022
(https://datahelpdesk.worldbank.org/knowledgebase/articles/906519-world-bank-country-and-lending-groups)

4. Covid19 and its impact on students by Kaggle
(https://www.kaggle.com/kunal28chaturvedi/covid19-and-its-impact-on-students)

In [ ]:
#import dataset/excel file

data1 = "../input/global-school-closures/global_school_closures_covid19.xlsx"
data2 = "../input/duration-school-closure-edited/duration_school_closures_edited (3).csv"
data3 = "../input/full-school-closure-evolution/full-school-closure-evolution-for-new-map-master-file-31082021.xlsx"
data4 = '../input/countries-enrollment-data/countries-enrollment-data-uis-sep-20.xlsx'
data5 = '../input/covid-dataset/owid-covid-data.csv'
data6 = "../input/country-class/CLASS_1.xlsx"
data7 = '../input/covid19-and-its-impact-on-students/COVID-19 Survey Student Responses.csv'


df1 = pd.read_excel(data1,'Mar19_Country-wide_school_Clos')
df1 = df1.iloc[1:,:]
df2 = pd.read_csv(data2)
df3 = pd.read_excel(data3, "Sheet1")
df4 = pd.read_excel(data4,'Sheet1')
df5 = pd.read_csv(data5)
df6 = pd.read_excel(data6, "List of economies")
df = pd.read_csv(data7)

In [ ]:
#Affected learner
df1.head()

In [ ]:
#School closures duration by country
df2.head()

In [ ]:
#School closures Evolution
df3.head()

In [ ]:
#Countries enrollment on education
df4.head()

In [ ]:
#Covid vaccinations
df5.head()

In [ ]:
#Countries classifications by income 2020-2021
df6.head()

In [ ]:
#COVID-19 Survey on India Students
df.head()

Description:
In this study, a cross-sectional survey is conducted with a sample size of 1182 students of different age groups from different educational institutions in Delhi National Capital Region (NCR).

In [ ]:
data = [44,133, 191, 814]
labels = ['Elementary School', 'Junior High School', 'Senior High School', 'College']

#define Seaborn color palette to use
colors = sns.color_palette('pastel')[0:5]

#create pie chart
plt.pie(data, labels = labels, colors = colors, autopct='%.0f%%')
plt.show()

# Data Preprocessing

In [ ]:
#extract vaccinations data by world
df5_world = df5[df5.location == 'World']

#got current Status of school closures, 2021-09-01
df5_country = df5[df5.date == "2021-09-01"]
df5_country

#Add the column "Month to dataframe"
df5_world['Month'] = pd.to_datetime(df5.date).dt.to_period("M")
df5_world

#got the continent of the country by alpha code
def continent(row):
     
    Str = row.ISO
    
    if Str == "SXM":
        return "NA"
    
    if Str== "TLS":
        return "AS"
    
    try:
        alpha2 = pc.country_alpha3_to_country_alpha2(Str)
    except:
        return row.Country
    
    try:
        answer = pc.country_alpha2_to_continent_code(alpha2)
    except:
        return row.Country
    
    return answer

# got the country classification by income from world bank data
def Class (data):
    iso = data.ISO
    
    for index, row in df6.iterrows():
        iso2 = row["Code"]
        
        if iso == iso2:
            return row["Income group"]
        
# got the country population density from vaccination data
def population_den (data):
    iso = data.ISO
    
    for index, row in df5_country.iterrows():
        iso2 = row["iso_code"]
        
        if iso == iso2:
            return row["population_density"]

# got the country population density from vaccination data        
def population (data):
    iso = data.ISO
    
    for index, row in df5_country.iterrows():
        iso2 = row["iso_code"]
        
        if iso == iso2:
            return row["population"]
        
# got the vaccinations rate by country from vaccination data         
def vaccine (data):
    iso = data.ISO
    
    for index, row in df5_country.iterrows():
        iso2 = row["iso_code"]
        
        if iso == iso2:
            
            answer = row.people_vaccinated_per_hundred
            
            return row["people_vaccinated_per_hundred"]
    #return "unk"
    
#appliead all function to make dataframe about current situationts and all additional variabel from different datasets
df3['Month'] = pd.to_datetime(df3.Date).dt.to_period("M")
df3["DATE"] = pd.to_datetime(df3["Date"])
df3_now = df3[df3["DATE"] == "2021-09-30"  ]
df3_now["continent"] = df3_now.apply(continent, axis=1)
df3_now["Class"] = df3_now.apply(Class, axis=1)
df3_now["Population"] = df3_now.apply(population, axis=1)
df3_now["Population dencity"] = df3_now.apply(population_den, axis=1)
df3_now["Vaccination in percent"] = df3_now.apply(vaccine, axis=1)

In [ ]:
#append column about student grades by age

# create a list of our conditions
conditions = [
    (df['Age of Subject'] <= 12),
    (df['Age of Subject'] >= 13) & (df['Age of Subject'] <= 15),
    (df['Age of Subject'] >= 16) & (df['Age of Subject'] <= 18),
    (df['Age of Subject'] >= 19)
    ]

# create a list of the values we want to assign for each condition
values = ['ES', 'JHS', 'SHS', 'College']

# create a new column and use np.select to assign values to it using our lists as arguments
df["Subject's Level of school"] = np.select(conditions, values )

#Converts Rating of Online Class Experience to the number and appendit as new column

Conditions = [
    df["Rating of Online Class experience"] == "Very poor",
    df["Rating of Online Class experience"] == "Poor",
    df["Rating of Online Class experience"] == "Average",
    df["Rating of Online Class experience"] == "Good",
    df["Rating of Online Class experience"] == "Excellent"
]
    
Value =[
    1,2,3,4,5
]


df["Rating of Online Class"] = np.select(Conditions, Value)


#Makes new column, Is student spent more time on Social Media than Self Study?
Conditions = [
    df["Time spent on social media"] > df["Time spent on self study"]
]

Value = [
    "YES"
]

df["Is student spent more time on Social Media than Self Study?"] =  np.select(Conditions,Value, default="NO")


#Makes new column, Is student have normal time of sleep?

Conditions = [
    (df["Subject's Level of school"] == "ES") & (  (df["Time spent on sleep"] >=9) & (df["Time spent on sleep"] <=12)  ) ,
    (df["Subject's Level of school"] == "JHS") & (  (df["Time spent on sleep"] >=8) & (df["Time spent on sleep"] <=10)  ) ,
    (df["Subject's Level of school"] == "SHS") & (  (df["Time spent on sleep"] >=8) & (df["Time spent on sleep"] <=10)  ) ,
    (df["Subject's Level of school"] == "College") & (  (df["Time spent on sleep"] >=7) & (df["Time spent on sleep"] <=10)  ) ,
    ]

Value = [
    "YES","YES","YES","YES"
]

df["Is student has normal time sleep?"] = np.select(Conditions,Value, default = "NO")


#
Conditions = [
    (df["Subject's Level of school"] == "ES") & (  (df["Time spent on sleep"] >=9) & (df["Time spent on sleep"] <=12)  ) ,
    (df["Subject's Level of school"] == "JHS") & (  (df["Time spent on sleep"] >=8) & (df["Time spent on sleep"] <=10)  ) ,
    (df["Subject's Level of school"] == "SHS") & (  (df["Time spent on sleep"] >=8) & (df["Time spent on sleep"] <=10)  ) ,
    (df["Subject's Level of school"] == "College") & (  (df["Time spent on sleep"] >=7) & (df["Time spent on sleep"] <=10)  ) ,
    ]

Value = [
    "YES","YES","YES","YES"
]

df["Total time of study"] = df["Time spent on Online Class"] + df["Time spent on self study"]

# Statistical Analysis and Data Visualization

# CHAPTER 1: COVID-19 and School Closures

# Where we stand today?

### Over 250 million students still affected by school closures

***Figure 1: World wide school closures status

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
ax.set(ylabel='countries')
#Set the colou pallete
sns.set_theme(style="whitegrid", font_scale = 1.5)
colors = ["#00FF00", "#FFFF00", "#FF0B04" , "#808080"]
sns.set_palette(sns.color_palette(colors))

data = [127,43, 17, 23]
labels = ['Fully open', 'Partially open', 'Closed due to COVID-19', 'Academic break']

#create pie chart
plt.pie(data, labels = labels, colors = colors, autopct='%.1f%%')
plt.show()

***Figure 2: School closures status by continents***

In [ ]:
df3_now1 = df3_now.groupby(by=["continent", "Status"], as_index = False).count()


fig, ax = plt.subplots(figsize=(12,12))
ax.set(ylabel='countries')
#Set the colou pallete
sns.set_theme(style="whitegrid", font_scale = 1.5)
colors = ["#00FF00", "#FFFF00", "#FF0B04" , "#808080"]
sns.set_palette(sns.color_palette(colors))


sns.barplot(ax = ax, x="Country", y = "continent", hue= "Status",hue_order = ["Fully open", "Partially open", "Closed due to COVID-19","Academic break"] , data = df3_now1).set_title("School closure status, by continent as of September 1, 2021")


**As September,1-2021, Majority of countries have fully opened schools (60.5 %), and 20.5 percent partially opened school, but there is still 8.1% countries have schools that were fully closed, mostly in Asia and North America (7 countries in asian and 8 countries in Norht America )**

In [ ]:
df4.sum()

***Figure 3: Affected student, September, 2-2021***

In [ ]:
N = 4

men = (24.312762 , 130.187042, 88.034918, 29.013427)
women = (26.119773, 143.471269, 93.663060, 28.455448) 

ind = np.arange(N)    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence
labels = ["Pre-primary", "Primary", "Secondary", "Tertiary"]

fig, ax = plt.subplots(figsize=(12,12))

ax.bar(labels, men, width, label='Men')
ax.bar(labels, women, width, bottom=men,
       label='Women')

plt.ylabel('Stundents (in million)')
plt.title("Affected student, September, 2-2021")
ax.legend()

plt.show()

**Over 250 million students still affected by school closures**

# The Evolutions of school closures

***Figure 4: The evolutions of scholl closures***

In [ ]:
df3_lab = df3.groupby(["DATE","Status"], sort = False, as_index=False).count()
df3_fig_all = df3_lab.pivot("DATE", "Status", "Country")

fig, ax = plt.subplots(figsize=(25, 10))
ax.tick_params(axis='x', rotation=90)
#Set the colou pallete
colors = ["#00FF00", "#FFFF00", "#FF0B04" , "#808080"]
sns.set_palette(sns.color_palette(colors))

sns.set_theme(style="whitegrid",  font_scale=1.5)

sns.lineplot(data=df3_fig_all,palette="tab10", linewidth=4).set_title("Time-series of school closure status from March 2020 to September 2021")

Figure 4 shows the patterns in school closures since March 2020. Right away, around 150 nations had completely shut their 

In the beginning of COVID-19, March to April, around 150 nations had completely shut their schools, while around 10 nations were to some extent shut and another 10 were completely open.

But right in September 2021,120 and more countries has openend their school

***Figure 5: School closures (Countries X days) evolutions by month***

In [ ]:
df3_fig1 = df3.groupby(["Month","Status"], sort = False, as_index=False).count()

fig, ax = plt.subplots(figsize=(25, 10))
ax.tick_params(axis='x', rotation=90)
#Set the colou pallete
colors = ["#00FF00", "#FFFF00", "#FF0B04" , "#808080"]
sns.set_palette(sns.color_palette(colors))

#sns.set_theme(style="whitegrid")

sns.barplot(data=df3_fig1,x="Month", y="Country", hue="Status" , linewidth=4).set_title("School closure evolution by month")
ax.set(ylabel='Countries X Days')

In [ ]:

start_date = "2020-02-03"
end_date = "2020-04-01"

start_date2 = "2020-04-01"
end_date2 = "2021-01-01"

start_date3 = "2021-01-01"


condition = (df3_lab['DATE'] > start_date) & (df3_lab['DATE'] <= end_date)
condition2 = (df3_lab['DATE'] > start_date2) & (df3_lab['DATE'] <= end_date2)
condition3 = (df3_lab['DATE'] > start_date3)

df3_1 = df3_lab.loc[condition]
df3_2 = df3_lab.loc[condition2]
df3_3 = df3_lab.loc[condition3]

df3_figure1 = df3_1.pivot("DATE","Status","Country")
df3_figure2 = df3_2.pivot("DATE","Status","Country")
df3_figure3 = df3_3.pivot("DATE","Status","Country")

### March 2020, is the beginning of world wide school closures

***Figure 6: The evolutions of school closures, February 2020 to March 2020***

In [ ]:
fig, ax = plt.subplots(figsize=(25, 10))

sns.set_theme(style="whitegrid", font_scale=1.5)

sns.lineplot(data=df3_figure1, palette="tab10", linewidth=4).set_title("The beginning of COVID-19")

In the begining of February , Mongolia started to closed their country due to COVID-19, While China still partially openend their education activities

February,21-2020, China started fully closed their schools activities

**End of the march, some countries (world wide) started to fully closed their country, that was when the trends of School Closure Starts**


## May to desember, reverse trend happened

***Figure 6: The evolutions of school closures, April 2020 to December 2020***

In [ ]:
fig, ax = plt.subplots(figsize=(25, 10))

sns.set_theme(style="whitegrid")

sns.lineplot(data=df3_figure2, palette="tab10", linewidth=4)

**May, where the trend was reversed**.The green line start to go up, that means some countries started to opened their school,

**After long academic break period (July 2020 to September 2020), the trend of Countries opened their schools was drastically stronger than before**, some countries joined to open their schools in the beginning of september 2020, and even more than 100 countries openend their school in the end of the 2020

## After long academic break period, school fully open trend even go up furthermore

***Figure 7: The evolutions of school closures, January 2021 to September 2021***

In [ ]:
fig, ax = plt.subplots(figsize=(25, 10))

sns.set_theme(style="whitegrid")

sns.lineplot(data=df3_figure3, palette="tab10", linewidth=4)

In April 2021, some countries was did academic break, it makes "Fully open" trend seems to be decreased, but it was not, arround 80 to 100 countries opened their schools in range February 2021- May 2021)

**After long acdemic break period, school open trend even GO UP FURTHERMORE**

# The relationship of school closures, with vaccinations, population,and economy class in current situations(September,01,2021)

In [ ]:
df3_now

## Countries income does not affect school closures in september 2021

***Figure 8: School closures by its country income calssification and status***

In [ ]:
df3_fig2 = df3_now.groupby(by = ["Status","Class"], as_index = False).count() 
df3_fig2 = df3_fig2[df3_fig2["Status"] != 'Academic break']

fig, ax = plt.subplots(figsize=(12,12))
#Set the colou pallete
sns.set_theme(style="whitegrid", font_scale = 1.5)
colors = ["#00FF00", "#FFFF00", "#FF0B04"]
sns.set_palette(sns.color_palette(colors))


sns.barplot(ax = ax, x="Country", y = "Class", hue= "Status" ,order = ["High income", "Upper middle income", "Lower middle income","Low income"], hue_order = ['Fully open', 'Partially open', 'Closed due to COVID-19'] ,data = df3_fig2).set_title("School closures, by countries income and status")

**Both in the lower and upper Economies categories, Each categories is looks the same**

In September 1-2021, Economy variable is no longer an obstacle, countries with low income, Mostly (Above 80 Percent) Opened their schools, as you can see in the figure 8

## Vaccinations rates does not affect school closures in september 2021 

***Figure 9: School closures by vaccinations rates***

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
#Set the colou pallete
sns.set_theme(style="whitegrid", font_scale = 1.5)
colors = ["#00FF00", "#FFFF00", "#FF0B04"]
sns.set_palette(sns.color_palette(colors))


sns.stripplot(ax = ax, y="Status", x = "Vaccination in percent"  ,data = df3_now).set_title("School closures, by vaccinations rates")

**In September, 1-2020, Countries with below 20% vaccination rate, still opened their schools**

**The five countries with 42% and up vaccinations rates, even Totally Closed their schools**

It seems Vaccinations rate are also  being ignored as a big indicator to deciding opened or closed schools in current situations

## Population density does not affect school closures in september 2021 

***Figure 10: School closures by population density***

In [ ]:
df3_figpol = df3_now[df3_now["Population dencity"] < 5000]

fig, ax = plt.subplots(figsize=(12,6))
#Set the colou pallete
sns.set_theme(style="whitegrid", font_scale = 1.5)
#colors = ["#00FF00", "#FFFF00", "#FF0B04"]
sns.set_palette(sns.color_palette(colors))


sns.stripplot(ax = ax, y="Status", x = "Population dencity"  ,data = df3_figpol).set_title("School closures, by population density")

## Population does not affect school closures in september 2021 

***Figure 10: School closures by population density***

In [ ]:
df3_figpop = df3_now[df3_now["Population"] < 400000000]

fig, ax = plt.subplots(figsize=(12,6))
#Set the colou pallete
sns.set_theme(style="whitegrid", font_scale = 1.5)
#colors = ["#00FF00", "#FFFF00", "#FF0B04"]
sns.set_palette(sns.color_palette(colors))


sns.stripplot(ax = ax, y="Status", x = "Population"  ,data = df3_figpop).set_title("School closures status, by Population")

**As we can see, the population also didn't counted as a big deal to decided school closures in current situations**

## The trend of world school closures against the trend of new cases, and the trend of new deaths

In [ ]:
df5_fig = df5_world.groupby(by=["Month"],as_index = False).sum()
df5_fig = df5_fig[df5_fig['Month'] >= "2021-01"  ]
df5_fig.people_vaccinated = df5_fig.people_vaccinated.div(1000000)
df5_fig

df5_fig2 = df5_world.groupby(by=["Month"],as_index = False).sum()
df5_fig2["new_deaths"] = df5_fig2["new_deaths"].div(1000000)

fig, ax = plt.subplots(figsize=(25, 10))
ax.tick_params(axis='x', rotation=90)
sns.set_theme(style="whitegrid", font_scale = 2)

sns.barplot(data=df5_fig2,x="Month", y="new_cases_per_million" , linewidth=4)

In [ ]:
fig, ax = plt.subplots(figsize=(25, 10))
ax.tick_params(axis='x', rotation=90)
sns.set_theme(style="whitegrid", font_scale = 2)

sns.barplot(data=df5_fig2,x="Month", y="new_deaths_per_million" , linewidth=4)

We noticed that in school closures evolution section, that september to the end year of 2020, that were when 120-125 countries opened their school, **the fact is, they  opened their school when the deaths and new cases rate, globally goes up**

It was unpredictable but also make sense, because it almost impossible to countries to pressing down the death and the new cases rate, but they try to saves their education

# Government policies are still the biggest factor of school closures evolutions, followed with Economy, Vaccinations and Population

## Observating countries with the most days of school closure

In [ ]:
#countries with the minimum school closures
df3_fig2 = df3[df3["Status"] == "Closed due to COVID-19"].groupby(by=["Country","Status"], as_index = False).count().sort_values(by="DATE", ascending= True).head(20)

#countries with the maximum school closures
df3_fig3 = df3[df3["Status"] == "Closed due to COVID-19"].groupby(by=["Country","Status"], as_index = False).count().sort_values(by="DATE", ascending= False).head(20)

bottom_closed_list = df3_fig2.Country.tolist()
top_closed_list = df3_fig3.Country.tolist()

bottom_closed = df3_now[df3.Country.isin(bottom_closed_list) ]
top_closed = df3_now[df3.Country.isin(top_closed_list) ]

### Countries with the most days of school closures by Vaccinations Rate (September, 1-2021)

***Figure 11: Countries with the most days of school closures by Vaccinations Rate***

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
#Set the colou pallete
sns.set_theme(style="whitegrid", font_scale = 1.5)
colors = ["#00FF00", "#FFFF00", "#FF0B04" , "#808080"]
sns.set_palette(sns.color_palette(colors))


sns.barplot(ax = ax, x="Vaccination in percent", y = "Country", hue= "Class", hue_order = ["High income", "Upper middle income", "Lower middle income","Low income"] , data = top_closed).set_title("Countries with the most days of school closures by vaccinations rates")

**Only one country with high income, which  closed its schools for long period**, 6/20 countries are middle upper economy class

**Countries with long school closures period and have good economy, all of them have good percentage of vaccinations rate in current time**

**some countries in this list even have no information about their vaccinations rate**

### Countries with the most days of school closures by Population

***Figure 12: Countries with the most days of school closures by population***

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
#Set the colou pallete
sns.set_theme(style="whitegrid", font_scale = 1.5)
colors = ["#00FF00", "#FFFF00", "#FF0B04" , "#808080"]
sns.set_palette(sns.color_palette(colors))


sns.barplot(ax = ax, x="Population", y = "Country", hue= "Class", hue_order = ["High income", "Upper middle income", "Lower middle income","Low income"] , data = top_closed).set_title("Countries with the most days of school closures by Population")

**Population is still big factors for school closures evolution, but in fact, there are some small countries with small population but still have long period of school closures**

**there are 7/20 countries with small population(below 70.000.000)** 

### Countries with the most days of school closures by Population density

***Figure 13: Countries with the most days of school closures by Population density***

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
#Set the colou pallete
sns.set_theme(style="whitegrid", font_scale = 1.5)
colors = ["#00FF00", "#FFFF00", "#FF0B04" , "#808080"]
sns.set_palette(sns.color_palette(colors))


sns.barplot(ax = ax, x="Population dencity", y = "Country", hue= "Class", hue_order = ["High income", "Upper middle income", "Lower middle income","Low income"] , data = top_closed).set_title("Countries with the most days of school closures by population density")

**5/20 countries have VERY big amount of population and population dencity**

Covid-19 is a virus, the virus sphread even faster when you meet other people, **having big population and big density, were obstacles and big problems for their school evolutions.**

**but for other countries with small amount of population and population density, they even still struggled and have long school closures period**

## Observating countries with the least days of school closure

***Figure 14: Countries with the most days of school closures by Vaccinations rates***

### Countries with the least days of school closures by Vaccinations rates

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
#Set the colou pallete
sns.set_theme(style="whitegrid", font_scale = 1.5)
colors = ["#00FF00", "#FFFF00", "#FF0B04" , "#808080"]
sns.set_palette(sns.color_palette(colors))


sns.barplot(ax = ax, x="Vaccination in percent", y = "Country", hue= "Class", hue_order = ["High income", "Upper middle income", "Lower middle income","Low income"] , data = bottom_closed).set_title("Countries with the least days of school closures by Vaccnations rates")

**ALL countries which have vaccinations information, they all have good vaccination rate and have High Income, good economy makes the countries move and act faster, they good economy also makes good vaccination rate, those factors accelerates their school closures evolutions**

### Countries with the least days of school closures by Population

***Figure 15: Countries with the most days of school closures by Population rates***

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
#Set the colou pallete
sns.set_theme(style="whitegrid", font_scale = 1.5)
colors = ["#00FF00", "#FFFF00", "#FF0B04" , "#808080"]
sns.set_palette(sns.color_palette(colors))


sns.barplot(ax = ax, x="Population", y = "Country", hue= "Class", hue_order = ["High income", "Upper middle income", "Lower middle income","Low income"] , data = bottom_closed).set_title("Countries with the least days of school closures by population")

**Having very Low population helping them to exelate their schools closures evolutions**, some of the countries even did not have population informations

**But, Even with very big population, japan still manages its country to openned the school**

### Countries with the least days of school closures by Populations density

***Figure 16: Countries with the most days of school closures by Population density***

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
#Set the colou pallete
sns.set_theme(style="whitegrid", font_scale = 1.5)
colors = ["#00FF00", "#FFFF00", "#FF0B04" , "#808080"]
sns.set_palette(sns.color_palette(colors))


sns.barplot(ax = ax, x="Population dencity", y = "Country", hue= "Class", hue_order = ["High income", "Upper middle income", "Lower middle income","Low income"] , data = bottom_closed).set_title("Countries with the least days of school closures by population dencity")

**Having very low population density helping them to excalate their schools closures evolutions**
**Singapore, Even with very high population density, Singapore still manages its country to openned the school**

# 

# CHAPTER 2: Covid-19 and its Impact on Student

# Current learning situations

## Time spent on online class

***Figure 17: Time spent on Online Class by level of school***

In [ ]:
#sns.set_theme(style="whitegrid", font_scale = 1.5)
sns.catplot(x="Subject's Level of school", y="Time spent on Online Class", kind="point" ,order=['ES','JHS','SHS','College'] ,height=5, aspect=2, data=df)
sns.catplot(x="Subject's Level of school", y="Time spent on Online Class", kind="violin" ,order=['ES','JHS','SHS','College'] , height=5, aspect=2, data=df)

The relationship between level of school and time spent on online class is unexpected, **College, which supposed to be the higher level of school, but it has lowest time spent of online class, even lower than elementary school level**

Looking at the graph, elementary schools has various (heterogen) distribution values, while the College student has homogen data, which means has less various values of Time spent on Online Class

in violin bar we can also see that the violin has long and thin shape, that's implied that schools did not has strong trend value of online learning, the student has many various amount of time on online Class.

It means each level of school did'nt have standard rule about learning hour to their students, each student has various time spent on online learning

## Rating on Online Class

***Figure 18: Rating on Online Class by level of school***

In [ ]:
sns.catplot(x="Subject's Level of school", y="Rating of Online Class", kind="point" ,order=['ES','JHS','SHS','College'] ,height=5, aspect=2, data=df)
sns.catplot(x="Subject's Level of school", y="Rating of Online Class", kind="violin" ,order=['ES','JHS','SHS','College'] ,height=5, aspect=2, data=df)

**The lower the school level, the higher the quality of online learning**

Elementary school student has best Online Class quality, based on time spent of Online class, and Rating of Online Class, and, College students has the worst Rating.

If we look at the graphs,the data is homogen, it means all groups of students has strong trend values.

College students has lowest time learning on Online Class, they also even has lowest Rating of online class experience

**it makes college student has worst Online Class quality, based on time spent of Online class, and Rating of Online Class**

## Time Spent on Self Study

***Figure 19: Time spent on self study by Level of school***

In [ ]:
sns.catplot(x="Subject's Level of school", y="Time spent on self study", kind="point" ,order=['ES','JHS','SHS','College'] ,height=5, aspect=2, data=df)
sns.catplot(x="Subject's Level of school", y="Time spent on self study", kind="violin" ,order=['ES','JHS','SHS','College'] ,height=5, aspect=2, data=df)

**Eventhough college's student has the worst online class experience, shortest time on online class, SO they pushed it and focused more on self study**

The relationships between level of school and time spent on online class is as we expected, **College is the higgest level of school, and it has the highest mean of time spent on online study**

the violin also showed that some college students has positive values trend, but the violin is thin, that's implied only small amount of students has very high time spent on self study.

## Total time of study

***Figure 20: Total time of Study by level of school***

In [ ]:
sns.catplot(x="Subject's Level of school", y="Total time of study", kind="point" ,order=['ES','JHS','SHS','College'] ,height=5, aspect=2, data=df)
sns.catplot(x="Subject's Level of school", y="Total time of study", kind="violin" ,order=['ES','JHS','SHS','College'] ,height=5, aspect=2, data=df)

**Even though college students has positive trend on Self Learning, College students still have the lowest mean of time study**

Colleges students got the maximum value of total time study, but that is only very small amount of people, they have small time to learning

**College and senior high school have various values of Total time of study**

Elementary school, has long and strong Trend of Total time of study, followed with junior high school,




# Students sleeping behavior

***Figure 21: Students sleeping behavior by level of School***

In [ ]:
sns.catplot(y="Is student has normal time sleep?",hue="Subject's Level of school",hue_order=['ES','JHS','SHS','College'],kind="count", height=12, data=df)


if we compared time spent on sleep on each level of school, **60% of elementary school's students have problem with their sleeping time, that's means only 40% of elementary students have normal time of sleep. it's kind of unexpected result**

**Surprisingly, College students have biggest percentage of Normal time of sleep**

**Junior high school and and senior high school also followed the positive trend on Sleep **

The American Academy of Sleep Medicine has recommended that children aged 6–12 years should regularly sleep 9–12 hours per 24 hours and teenagers aged 13–18 years should sleep 8–10 hours per 24 hours

Source: https://www.cdc.gov/healthyschools/features/students-sleep.htm#:~:text=The%20American%20Academy%20of%20Sleep,10%20hours%20per%2024%20hours.

# Students prefer to self study

***Figure 22: Is student spent more time on Social Media than Self Study?***

In [ ]:
sns.catplot(y="Is student spent more time on Social Media than Self Study?",hue="Subject's Level of school",kind="count",height = 10 ,data=df)

**Students on each level of school has strong positive trend on preferred to spent their time on self study, rather than on social media, which means this is very positives for students**

# Conclusion

There are still many countries with many obstacles that have opened school activities. These obstacles, as already discussed, are the low percentage of vaccination, the large population and overcrowding, and the poor economy which makes the situation worse.

The only thing that made them open schools was their concern for education.

As time goes on, they learn and try to open their schools

This long school closure has an impact on all levels of students, even college students, they have a bad learning experience, some students 

A small number of students have benefited from this pandemic, such as a lot of study time, focus on themselves studying on the internet, unfortunately a large part of these students, seem to have lost their enthusiasm for learning

Contact me at:
email: gusanwaakbar@gmail.com
ig: @gusanwaakbar